# **Caso: ¿Quién sobrevive al Titanic?**

Todos los datos utilizados en esta clase se pueden encontrar en la siguiente competencia de Kaggle: **https://www.kaggle.com/c/titanic**

# Entendimiento del negocio

- El hundimiento del Titanic es uno de los accidentes náuticos más conocidos de la historia. 
- Fallecieron **1,502** personas de los **2,224** pasajeros. 
- Una de las principales causas es que no existían botes salvavidas suficientes para los pasajeros y la tripulación.
- Descubriremos algunos **factores relevantes** para la supervivencia de los pasajeros
- Nuestro objetivo es obtener una **probabilidad de supervivencia** y podremos responder a la siguiente pregunta: ¿hubiéramos sobrevivido en el Titanic?

## Comprensión de los datos

Leemos los datos (train y test) a partir de archivos CSV(Comma Separated Values).

In [0]:
import pandas as pd
train = pd.read_csv('https://raw.githubusercontent.com/javalillo13689/hackspace/master/titanic_train.csv')
test = pd.read_csv("https://raw.githubusercontent.com/javalillo13689/hackspace/master/titanic_test.csv")

Visualizamos el tamaño de la data contenida en ambos archivos a través del método **shape**

In [0]:
train.shape, test.shape

A fin de comprobar que la data ha sido importada de forma correcta ejecutamos el método **head** para train y test por separado.

In [0]:
train.head(3)

In [0]:
test.head(3)

Mostrar el tipo de datos de las variables

In [0]:
train.dtypes

# Preprocesamiento de los datos

##Preprocesamiento de nulos o vacíos

Mostrar las columnas con datos faltantes y el porcentaje de los mismos

In [0]:
def valores_perdidos (tabla): 
        total_nulos = tabla.isnull().sum()
        porcentaje_nulos = 100 * tabla.isnull().sum()/len(tabla)
        tabla_nulos = pd.concat([total_nulos, porcentaje_nulos], axis=1)
        tabla_nulos_nombres = tabla_nulos.rename(columns = {0 : 'total_nulos', 1 : '% nulos'})
        return tabla_nulos_nombres [tabla_nulos_nombres['total_nulos'] != 0].sort_values('total_nulos', ascending=False)

In [0]:
valores_perdidos(train)

In [0]:
valores_perdidos(test)

**ms.matrix** (librería missingno): Grafica cada columna dejando espacios donde aparezcan valores perdidos.

Es más intuitivo y fácil de mapear, pero es dificil de visualizar si contamos con muchas columnas

In [0]:
import missingno as ms
ms.matrix(train)

In [0]:
ms.matrix(test)

Reemplazamos los datos faltantes según el tipo de dato en cada columna:

1.   Si el tipo de dato es **numérico** (int64, float64) procedemos a reemplazarlo por la **media**
2.   Si el tipo de dato **no es numérico** (object) procedemos a reemplazarlo por la **moda**



In [0]:
train['cabina'].fillna(train['cabina'].mode()[0], inplace=True)
train['edad'].fillna(train['edad'].mean(), inplace=True)
train['puerto_embarque'].fillna(train['puerto_embarque'].mode()[0], inplace=True)

In [0]:
test['cabina'].fillna(train['cabina'].mode()[0], inplace=True)
test['edad'].fillna(train['edad'].mean(), inplace=True)
test['tarifa'].fillna(train['tarifa'].mean(), inplace=True)

In [0]:
valores_perdidos(test)

##Preprocesamiento de outliers o valores fuera de lo normal

1.   Realizamos un gráfico de boxplot para identificar valores fuera de la dispersión normal de la data. **Este método solo se aplica a variables numéricas**
2.   Reemplazaremos estos valores, si lo hubiera, por el **percentil 95**

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.boxplot(train['edad'], orient="v")
plt.show()
sns.boxplot(train['tarifa'], orient='v')
plt.show()

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.boxplot(test['edad'], orient="v")
plt.show()

In [0]:
outliers_a_reemplazar = train[train['edad'] > train['edad'].quantile(.95)].index
train.loc[outliers_a_reemplazar,'edad'] = train['edad'].quantile(.95)

In [0]:
outliers_a_reemplazar = train[train['tarifa'] > train['tarifa'].quantile(.95)].index
train.loc[outliers_a_reemplazar,'tarifa'] = train['tarifa'].quantile(.95)

In [0]:
outliers_a_reemplazar = test[test['edad'] > test['edad'].quantile(.95)].index
test.loc[outliers_a_reemplazar,'edad'] = test['edad'].quantile(.95)

##Preprocesamiento de las variables o *Feature Engineering*

1.   Evaluamos según la lógica del problema **qué variables podrían impactar directamente en nuestro objetivo a predecir**. En este caso la supervivencia.
2.   Reemplazaremos estas variables, si lo hubiera, por valors **numéricos** que ayuden al modelo a brindarle mayor importancia para la predicción

In [0]:
sns.swarmplot(x='sobrevivio',y='edad',data=train,hue='sexo')

In [0]:
sns.countplot(x = 'puerto_embarque', data = train[train['sobrevivio'] == 0])

In [0]:
palabras_a_numeros = {"puerto_embarque":  {"S": 0, "Q":1, "C": 2},"sexo": {"male":0,"female":1}}

In [0]:
train.replace(palabras_a_numeros, inplace = True)

In [0]:
test.replace(palabras_a_numeros, inplace = True)

In [0]:
test.head(3)

##Preprocesamiento de distribuciones que se alejan de la distribución normal

1.   Seleccionamos aquellas variables **numéricas** que presenten un comportamiento alejado a la distribución normal.
2.   Estandarizamos estas variables, si lo hubiera, para uniformizar la data lista para ingresar al modelo.

In [0]:
sns.distplot(train['tarifa'])
plt.show()
sns.distplot(train['edad'])
plt.show()

Transformación BoxCox

In [0]:
from scipy.special import boxcox1p
train['tarifa'] = boxcox1p(train['tarifa'], 0.15)
sns.distplot(train['tarifa'])

Transformación Logarítmica

In [0]:
import numpy as np
sns.distplot(np.log1p(train['tarifa']))

In [0]:
test['tarifa'] = boxcox1p(test['tarifa'], 0.15)

# **Modelamiento**

En esta sesión ejecutamos nuestros primeros modelos de machine learning. Para el caso del titanic utilizamos **modelos de clasificación** porque vamos a predecir una **clase** o característica: 1=sobrevivió, 0=no sobrevivió.

Los modelos que ejecutaremos serán:

1.   Regresión Logística
2.   Árbol de clasificación (y también Random Forest)

## Seleccionando las variables predictoras

Filtramos cada base de datos (train y test) para quedarnos únicamente con las variables predictoras que utilizaremos para nuestro modelo.

Por ello descartamos de esta selección a las siguientes variables:

1.   **id** porque el lugar que ocupa actualmente en la lista de registro no influye en la supervivencia del pasajero
2.   **nombre** porque el nombre del pasajero no es relevante para el modelo
3.   **ticket** porque el ticket de compra que consiguió cada pasajero no es decisivo para la supervivencia del pasajero
4.   **cabina** porque el nombre de la habitación donde estuvo el pasajero no es relevante para el modelo

In [0]:
train.drop(['id','nombre','ticket','cabina'],axis=1,inplace=True)
test.drop(['id','nombre','ticket','cabina'],axis=1,inplace=True)

##Dividiendo la base de train en x_train, y_train

1.   Para comenzar con la ejecución del modelo almacenaremos solo  **las variables predictoras** en X
2.   La variable objetivo (**target**) que para el titanic es la columna **sobrevivio** la almacenamos en y

In [0]:
X = train.drop('sobrevivio',axis=1)
y = train['sobrevivio']

In [0]:
#Todo modelo supervisado requiere dividir la data entre variables predictoras y variable objetivo (target)
from sklearn.model_selection import train_test_split
x_train,x_valid,y_train, y_valid=train_test_split(X,y)

##Aplicando el modelo de Regresión Logística sobre train

In [0]:
#Importamos el modelo LogisticRegression de la librería linear_model y lo almacenamos en la variable logreg
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

In [0]:
#Entrenamos el modelo con la variables predictoras (x_train) y el target (y_train)
logreg.fit(x_train, y_train)

In [0]:
#Obtenemos las predicciones utilizando nuestro modelo entrenado con las variables predictoras (x_valid)
logreg_pred = logreg.predict(x_valid)
logreg_pred

##Obteniendo las probabilidades de predicción con la Regresión Logística

Recuerda, **predict_proba** nos va a decir la **probabilidad** de que el registro sea 0 ó 1. Para el caso titanic **si sobreviviría o no**.

Esta función requiere solo 1 parámetro: la lista de **variables predictoras** para generar la probabilidad de cada registro

In [0]:
#Ejecutamos el parámetro interno "predict_proba" del modelo logreg
probabilidades = logreg.predict_proba(x_valid)
probabilidades

##Obteniendo el Accuracy Score de la Regresión Logística

Recuerda que la función **accuracy_score** nos va a decir el porcentaje de predicción de nuestro modelo, es decir, **cuánto del total** de registros nos está **prediciendo bien**.

Esta función requiere dos (2) parámetros: la lista de valores en **target** y la lista de **predicciones** hechas por nuestro modelo.

In [0]:
#Importamos la métrica accuracy_score de la librería metrics
from sklearn.metrics import accuracy_score

In [0]:
#Calculamos la métrica del modelo de clasificación: accuracy_score. Con sintaxis de Python fijamos 4 decimales
print('El accuracy para mi primero modelo es :{0:.4f}'.format(accuracy_score(y_valid,logreg_pred)))

##Obteniendo la Matriz de Confusión de la Regresión Logística

In [0]:
#Importamos el método confusion_matrix de la librería metrics
from sklearn.metrics import confusion_matrix

Recuerda, **conf_matrix** nos va a decir la **cantidad de registros** almacenados en cada uno de los siguientes escenarios:

1.   **True Positive**: el target real y la predicción coinciden en 1. Para el caso de titanic **predice de manera correcta la supervivencia**
2.   **False Positive**: el target real y la predicción coinciden en 0. Para el caso de titanic **predice de manera correcta la muerte**
3.   **True Negative**: el target real=1 y la predicción=0. Para el caso de titanic **predice de manera incorrecta la supervivencia**
2.   **False Negative**: el target real=0 y la predicción=1. Para el caso de titanic **predice de manera incorrecta la muerte**

In [0]:
#Entrenamos la matriz de confusión y almacenamos el resultado en la variable conf_matrix
conf_matrix = confusion_matrix(y_valid,logreg_pred)

In [0]:
#Utilizamos el método heatmap de la librería seaborn para dibujar la matriz de confusión
import seaborn as sns
sns.heatmap(conf_matrix, annot = True, annot_kws={"size": 18}, fmt=".1f")

##Aplicando el modelo de Árbol de Clasificación sobre train

In [0]:
#Importamos el modelo DecisionTreeClassifier de la librería tree y lo almacenamos en la variable tree_clf
from sklearn.tree import DecisionTreeClassifier
tree_clf = DecisionTreeClassifier()

In [0]:
#Entrenamos el modelo con la variables predictoras (x_train) y el target (y_train)
tree_clf.fit(x_train, y_train)

In [0]:
#Obtenemos las predicciones utilizando nuestro modelo entrenado con las variables predictoras (x_valid)
tree_pred = tree_clf.predict(x_valid)
tree_pred

In [0]:
#Calculamos la métrica del modelo de clasificación: accuracy_score. Con sintaxis de Python fijamos 4 decimales
print('El accuracy para mi primero modelo es :{0:.4f}'.format(accuracy_score(y_valid,tree_pred)))

##Optimización de hiperparámetros del Árbol de Clasificación sobre train

In [0]:
#Definimos una función que itere valores de 1 a 10 para el nivel de profundidad del árbol
for i in range(1,10):
    tree_clf = DecisionTreeClassifier(max_depth=i)
    tree_clf.fit(x_train,y_train)
    y_pred = tree_clf.predict(x_valid)
    print("Mi arbol da un accuracy de:", accuracy_score(y_valid,y_pred), "cuando su max_depth es: ", i)

Una vez ejecutada la función podemos visualizar que con **tres niveles de profundidad** se obtiene el **más alto accuracy**

Ahora vamos a **actualizar el modelo** utilizando el **parámetro max_depth**

In [0]:
#Actualizamos el modelo tree_clf ingresando un nuevo parámetro: max_depth=3
tree_clf = DecisionTreeClassifier(max_depth=3)
tree_clf.fit(x_train,y_train)
tree_pred = tree_clf.predict(x_valid)

##Aplicando el modelo de Random Forest sobre train

In [0]:
#Importamos el modelo RandomForestClassifier de la librería ensamble y lo almacenamos en la variable rnd_clf
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(n_estimators = 1000, n_jobs = -1,max_depth=3)

In [0]:
#Entrenamos el modelo con la variables predictoras (x_train) y el target (y_train)
rnd_clf.fit(x_train, y_train)

In [0]:
#Obtenemos las predicciones utilizando nuestro modelo entrenado con las variables predictoras (x_valid)
rnd_pred = tree_clf.predict(x_valid)
rnd_pred

In [0]:
#Calculamos la métrica del modelo de clasificación: accuracy_score. Con sintaxis de Python fijamos 4 decimales
print('El accuracy para mi primero modelo es :{0:.4f}'.format(accuracy_score(y_valid,rnd_pred)))

##Importancia de variables del Random Forest

In [0]:
import numpy as np
importances = rnd_clf.feature_importances_
cols = x_train.columns
plt.figure(figsize=(8,12))
indices = np.argsort(importances)
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], align = 'center')
plt.yticks(range(len(indices)),[cols[i] for i in indices])
plt.show()